In [1]:
import tensorflow
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
import numpy as np
from numpy.linalg import norm
import os
from tqdm import tqdm
import pickle
from PIL import Image

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [3]:
model = ResNet50(weights='imagenet',include_top=False,input_shape=(224,224,3))
model.trainable = False
model = tensorflow.keras.Sequential([
    model,
    GlobalMaxPooling2D()
])

2022-04-23 21:16:52.672077: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def extract_features(img_path,model):
    img = image.load_img(img_path,target_size=(224,224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / norm(result)
    
    return normalized_result

In [5]:
filenames = []

In [6]:
def getListOfFiles(dirName):
    listOfFile=os.listdir(dirName)
    allFiles=[]
    for entry in listOfFile:
        fullpath=os.path.join(dirName,entry)
        if os.path.isdir(fullpath):
            allFiles=allFiles+getListOfFiles(fullpath)
        else:
            allFiles.append(fullpath)
    
    return allFiles

In [7]:
dirname="img"
# extracting image features
filenames=getListOfFiles(dirname)
feature_list = []

In [8]:
filenames[0]

'img/Marled_Knit_Racerback_Tank/img_00000002.jpg'

In [ ]:
for file in tqdm(filenames):
    x=file.split(".")
    if x[-1]=="jpg" or x[-1]=="png": 
        feature_list.append(extract_features(file,model))

  0%|▏                                 | 1164/289221 [02:41<10:35:07,  7.56it/s]

In [13]:
pickle.dump(feature_list,open('embeddings.pkl','wb'))
pickle.dump(filenames,open('filenames.pkl','wb'))